# Global Food Energy Grid Methods Notebook
### This notebook corresponds to the methods for the paper. It is broken into 3 sections.
### 1. National data processing, using csv and json data to build pandas dataframes containing global and national food data.
### 2. Grid data processing. National and global data is distributed onto a 1-degree resolution global grid proportionally to local surrogate data. The final result is an xarray dataset.
### 3. Plots found in the paper so the data can be visualized and analyzed.

## 1. Tabular Data Processing

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import random
import matplotlib.colors as mpl_colors
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import json
import sys
import pandas as pd
import geopandas as gpd

with open('config.json','r') as f:
    config = json.load(f)

with open(config['paths']['Names'],'r') as f:
    Names = json.load(f)

fbs_labels_df = pd.read_csv(config['paths']['fbs_labels'])
fbs_labels_df = fbs_labels_df[[col for col in fbs_labels_df.columns if col != 'Item']]

fbs_df = pd.read_csv(config['paths']['fbs'], encoding='ISO-8859-1')
fbs_df = pd.merge(fbs_df, fbs_labels_df, on='Item Code', how='left')
original_food_items = fbs_df['Item'].unique()

### Remove regions that contain multiple countries

In [2]:
fbs1 = fbs_df[~fbs_df['Class'].isin(['DROP', 'Aggregate'])]

multi_ctry_regions = ['Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'China',
 'Caribbean', 'Central America', 'Central Asia', 'Eastern Africa',
 'Eastern Asia', 'Eastern Europe', 'Europe', 'European Union (27)',
 'Land Locked Developing Countries', 'Least Developed Countries',
 'Low Income Food Deficit Countries', 'Melanesia', 'Micronesia', 'Middle Africa',
 'Net Food Importing Developing Countries', 'Netherlands Antilles (former)',
 'Northern Africa', 'Northern America', 'Northern Europe', 'Oceania',
 'Polynesia', 'Small Island Developing States', 'South America',
 'South-eastern Asia', 'Southern Africa', 'Southern Asia', 'Southern Europe',
 'Western Africa', 'Western Asia', 'Western Europe', 'World']
special_groups = ['European Union (27)', 'Land Locked Developing Countries', 'Least Developed Countries', 'Low Income Food Deficit Countries', 'Net Food Importing Developing Countries', 'Small Island Developing States']
regions = ['Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', 'Central Asia', 'Eastern Africa', 'Eastern Asia', 'Eastern Europe', 'Europe', 'Melanesia', 'Micronesia', 'Middle Africa', 'Northern Africa', 'Northern America', 'Northern Europe', 'Oceania', 'Polynesia', 'South America', 'South-eastern Asia', 'Southern Africa', 'Southern Asia', 'Southern Europe', 'Western Africa', 'Western Asia', 'Western Europe', 'World']
#[i for i in multi_ctry_regions if i not in special_groups + regions] # checking special groups and regions not listed

fbs1 = fbs1[~fbs1['Area'].isin(multi_ctry_regions)]

food21 = fbs1[fbs1['Element'] == 'Food']['Y2021'].sum()
impo21 = fbs1[fbs1['Element'] == 'Import Quantity']['Y2021'].sum()
print('Percent of imports out of total food supply in 2021:', str(round(impo21/food21 * 100,2)) + '%.')

s1 = "The FAO estimates that there are {:e} calories in the global food supply".format(fbs_df.query('Area == "World" and Item == "Grand Total" and Element == "Food supply (kcal)"')['Y2015'].item())
s2 = "However summing over all of the countries (except for China nad the former Netherlands Antilles, which are double counted by other nations/ subnational regions) gives fewer calories: {:e}".format(fbs1[fbs1['Element'] == 'Food supply (kcal)']['Y2015'].sum().item())
s3 = "This is a global difference of approximately {:e} percent".format(100 * fbs_df.query('Area == "World" and Item == "Grand Total" and Element == "Food supply (kcal)"')['Y2015'].item() / fbs1[fbs1['Element'] == 'Food supply (kcal)']['Y2015'].sum().item())
print(s1,'\n',s2,'\n',s3)

Percent of imports out of total food supply in 2021: 35.96%.
The FAO estimates that there are 7.787052e+09 calories in the global food supply 
 However summing over all of the countries (except for China nad the former Netherlands Antilles, which are double counted by other nations/ subnational regions) gives fewer calories: 7.779255e+09 
 This is a global difference of approximately 1.001002e+02 percent


### Group FBS by area, item, and element, sum over other values, and pivot by element. New columns: 
Country, Item, Production, Feed, Processing, Food, Food Supply (kcal)      

In [3]:
fbs=fbs1.pivot(index = ['Area', 'Item','Class','Type','kcal','Processed'], columns="Element", values="Y2015").reset_index()
fbs=fbs[['Area', 'Item', 'Class', 'Type', 'kcal', 'Processed', 'Domestic supply quantity', 'Export Quantity', 'Losses',
 'Fat supply quantity (t)', 'Feed', 'Food supply (kcal)', 'Import Quantity', 'Food',
 'Other uses (non-food)', 'Processing', 'Production','Protein supply quantity (t)',
 'Residuals', 'Seed', 'Stock Variation','Tourist consumption','Food supply quantity (kg/capita/yr)']]
fbs = fbs.groupby(["Item","Area"]).agg({
    col: ('sum' if col not in ["Item", "Class", "Type", "kcal", "Processed"] else 'first') for col in fbs.columns
})
fbs = fbs.rename_axis(None, axis=1).drop(columns=['Item','Area']).reset_index() # Reset Index so Item and Area are just columns

### Adjust Production based on Import-Export Difference

In [4]:
diff = fbs['Import Quantity'].sum() - fbs['Export Quantity'].sum()
tot  = (fbs['Import Quantity'].sum() + fbs['Export Quantity'].sum()) / 2
print(f'The FAO Food Balance Sheet total import Quantity include approximately {round(diff / 1000)} more Megatons of food than the total export quantity, which is approximately {round(diff/tot * 100)}% of global food trade.')

grouped = fbs.groupby('Item').agg({'Production': 'sum', 'Import Quantity': 'sum', 'Export Quantity': 'sum'})
grouped['import-export-factor'] = (grouped['Production'] + grouped['Import Quantity'] - grouped['Export Quantity']) / grouped['Production']
df = pd.merge(fbs, grouped['import-export-factor'], how='left', on='Item')
fbs['AdjustedProd'] = df['Production'] * df['import-export-factor']

foods = [item for item in fbs.Item.unique()]
sua_balance={}
print('\nFoods with more than 20 MegaTonnes unacounted production or consumption (units 1000 tonnes)')
for food in foods:
    df = fbs[fbs['Item'] == food].sum()
    sua_balance[food] = df['Production'] + df['Import Quantity'] - df['Export Quantity'] - df['Stock Variation'] - (df['Food'] + df['Feed'] + df['Seed'] + df['Losses'] + df['Processing'] + df['Other uses (non-food)'] + df['Tourist consumption'] + df['Residuals'])
    sua_balance[food] = sua_balance[food].item()
    try:
        assert(abs(sua_balance[food]) < 20)
    except:
        print(food, sua_balance[food])

The FAO Food Balance Sheet total import Quantity include approximately 228 more Megatons of food than the total export quantity, which is approximately 14% of global food trade.

Foods with more than 20 MegaTonnes unacounted production or consumption (units 1000 tonnes)
Apples and products 22.0
Beans 26.0
Maize and products 21.0
Oilcrops Oil, Other 23.0
Oilcrops, Other 22.0
Roots, Other 20.0
Wheat and products 31.0


### Put Population back in to the fbs

In [5]:
fbs_pop = fbs_df[fbs_df['Element'] == 'Total Population - Both sexes'].copy()
fbs_pop['pop2015'] = fbs_pop['Y2015'] * 1000
fbs_pop['pop2021'] = fbs_pop['Y2021'] * 1000
fbs = fbs.merge(fbs_pop[['Area', 'pop2015','pop2021']], on='Area', how='left')

### Separate Production into Crop, Animal, and Processed based on Food Type and Convert Production to Energy

In [6]:
fbs['kcalpkg'] = 10 * fbs['kcal']
fbs['kcalpkg'] = fbs['kcalpkg'].fillna(fbs['Food supply (kcal)'] / fbs['Domestic supply quantity']).replace([np.inf, -np.inf], np.nan)

fbs['CropProdMCal'] = fbs.apply(lambda r: r['AdjustedProd']*r['kcalpkg'] if r['Class'] == 'Crop' and r['Processed'] != 'Yes' else 0, axis=1)
fbs['AnimProdMCal'] = fbs.apply(lambda r: r['AdjustedProd']*r['kcalpkg'] if r['Class'] in ['Livestock', 'Seafood'] and r['Processed'] != 'Yes' else 0, axis=1)
fbs['ProcProdMCal'] = fbs.apply(lambda r: r['AdjustedProd']*r['kcalpkg'] if r['Processed'] == 'Yes' else 0, axis=1)

MCalpCalpCappDay = 10**6 / (7.4 * 10**9 * 365)
print('CropProd total', fbs.CropProdMCal.sum() * MCalpCalpCappDay)
print('AnimProd total', fbs.AnimProdMCal.sum() * MCalpCalpCappDay)
print('ProcProd total', fbs.ProcProdMCal.sum() * MCalpCalpCappDay)

# Non-Production Columns
for fbs_col in ['Feed', 'Food', 'Processing', 'Losses', 'Residuals', 'Seed', 'Stock Variation',
                'Tourist consumption', 'Import Quantity', 'Export Quantity']:
    new_col = fbs_col[:4] + 'MCal'
    fbs[new_col] = fbs.apply(lambda r: r[fbs_col]*r['kcalpkg'], axis=1)
    print(new_col, 'total', fbs[new_col].sum() * MCalpCalpCappDay)

# Other Unnacounted Utilization and Loss
fbs['FoodSupplyMCal'] = fbs['Food supply (kcal)']
fbs['OtherMCal'] = fbs['CropProdMCal'] + fbs['AnimProdMCal'] + fbs['ProcProdMCal'] - fbs['FeedMCal'] - fbs['ProcMCal'] - fbs['FoodSupplyMCal']

print('Other    total', fbs.OtherMCal.sum() * MCalpCalpCappDay)
print('FoodSupp total', fbs.FoodSupplyMCal.sum() * MCalpCalpCappDay)
print()

for item in fbs.Item.unique():
    df = fbs[fbs['Item'] == item]
    total_prod = sum([df[col].sum() for col in ['CropProdMCal', 'ProcProdMCal', 'AnimProdMCal']])
    total_cons = sum([df[col].sum() for col in ['FoodSupplyMCal', 'FeedMCal', 'ProcMCal', 'OtherMCal']])
    epsilon = 1
    try:
        assert(abs(total_prod - total_cons) < epsilon)
    except:
        print('Unbalanced:',item, round(total_prod), round(total_cons))
        
print('\nItems with biggest discrepency between Food column converted to MCal and FoodSupply columns given already in Calories units')
data = {}
for item in fbs.Item.unique():
    df = fbs[fbs['Item'] == item]
    data[item] = {'FoodMCal':df['FoodMCal'].sum(), 'FoodSupplyMCal': df['FoodSupplyMCal'].sum()}

df = pd.DataFrame(data)
df = df.transpose()
df['diff'] = df['FoodMCal'] - df['FoodSupplyMCal']
df['diff_ratio'] = df['diff'] / df['FoodSupplyMCal']
df['ratio'] = abs(df['FoodMCal'] / df['FoodSupplyMCal'])
df.sort_values(by='ratio').head(20)

CropProd total 6078.612043687523
AnimProd total 533.8167110579926
ProcProd total 1253.47210602959
FeedMCal total 1410.461313159608
FoodMCal total 3317.8860093053813
ProcMCal total 1712.503391336542
LossMCal total 310.92326590638805
ResiMCal total 61.60389152896229
SeedMCal total 135.67322462207684
StocMCal total 133.2923429826451
TourMCal total 4.422078429280303
ImpoMCal total 1642.4878983178066
ExpoMCal total 1561.248460595916
Other    total 1862.803483487397
FoodSupp total 2880.138754598297

Unbalanced: Aquatic Animals, Others 638154 638176
Unbalanced: Beverages, Alcoholic 74037510 74050426
Unbalanced: Beverages, Fermented 10145379 10148581
Unbalanced: Cephalopods 2625191 2625209
Unbalanced: Crustaceans 7387426 7387430
Unbalanced: Demersal Fish 9350838 9350840
Unbalanced: Fish, Liver Oil 32903 33156
Unbalanced: Freshwater Fish 42534512 42534519
Unbalanced: Marine Fish, Other 6153213 6153214
Unbalanced: Molluscs, Other 4273941 4273943

Items with biggest discrepency between Food colum

,FoodMCal,FoodSupplyMCal,diff,diff_ratio,ratio
Pimento,8.845000e+05,1.131073e+07,-1.042623e+07,-0.921800,0.078200
Cottonseed,5.060000e+03,1.273192e+04,-7.671920e+03,-0.602574,0.397426
"Citrus, Other",2.981160e+06,7.089659e+06,-4.108499e+06,-0.579506,0.420494
"Lemons, Limes and products",2.509200e+06,4.298990e+06,-1.789790e+06,-0.416328,0.583672
Onions,1.951824e+07,3.179883e+07,-1.228059e+07,-0.386196,0.613804
Sunflower seed,1.232000e+06,1.880565e+06,-6.485645e+05,-0.344878,0.655122
Grapefruit and products,1.293600e+06,1.845282e+06,-5.516820e+05,-0.298969,0.701031
"Meat, Other",5.428500e+06,7.638467e+06,-2.209967e+06,-0.289321,0.710679
"Fish, Body Oil",4.568879e+05,6.344728e+05,-1.775849e+05,-0.279894,0.720106
Cream,6.257550e+06,8.103902e+06,-1.846352e+06,-0.227835,0.772165


### Get Livestock Data which isn't in the FBS

In [7]:
with open(config['paths']['fao_regions.json']) as json_file:
    fao_regions = json.load(json_file)
fao_country_to_region = {country: region for region, countries in fao_regions.items() for country in countries}

# Source: https://www.fao.org/3/i2294e/i2294e00.pdf
lsu_data = {
    'Region': ['Near East North Africa', 'North America', 'Africa South of Sahara', 'Central America', 'South America', 'South Africa', 'OeCD', 'East and South East Asia', 'South Asia', 'Transition Markets', 'Caribbean', 'Near East', 'Other'],
    'Cattle': [0.70, 1.00, 0.50, 0.70, 0.70, 0.70, 0.90, 0.65, 0.50, 0.60, 0.60, 0.55, 0.60],
    'Buffalo': [0.70, np.nan, np.nan, np.nan, np.nan, np.nan, 0.70, 0.70, 0.50, 0.70, 0.60, 0.60, 0.60],
    'Sheep': [0.10, 0.15, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10],
    'Goats': [0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10],
    'Swine / pigs': [0.20, 0.25, 0.20, 0.25, 0.25, 0.20, 0.25, 0.25, 0.20, 0.25, 0.20, 0.25, 0.20],
    'Asses': [0.50, 0.50, 0.30, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50],
    'Horses': [0.40, 0.80, 0.50, 0.50, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.56, 0.65],
    'Mules and hinnies': [0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60],
    'Camels': [0.75, np.nan, 0.70, np.nan, np.nan, np.nan, 0.90, 0.80, np.nan, np.nan, np.nan, 0.70, np.nan],
    'Chickens': [0.01, np.nan, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
}

lsu_df = pd.DataFrame(lsu_data)
lsu_df.set_index('Region', inplace=True)

# Source: FAOSTAT CropsAndLivestockProducution
livestock_df = pd.read_csv(config['paths']['fao_livestock.csv'])
livestock_df = livestock_df.replace({'Area': Names})
livestock_df.rename(columns={'Area': 'ISO3'}, inplace=True)
livestock_df = livestock_df[livestock_df['Item'].isin(livestock_df[livestock_df['Element'] != 'Production'].Item.unique())]

# Use FAO region categoriations along with regional lsu coefficients to get lsu values for each country
livestock_df['FAO_Region'] = livestock_df['ISO3'].map(fao_country_to_region)
livestock_df['LSU_coeff'] = np.nan
valid_rows = livestock_df['Item'].isin(lsu_data.keys())

def get_lsu_value(row):
    try:
        val = lsu_df.at[row['FAO_Region'], row['Item']]
        if np.isnan(val):
            return lsu_df[row['Item']].mean()
        else:
            return val
    except KeyError:
        return np.nan 
    
# Format the livestock counts dataframe to have consistent units and be ready for dasymetric mapping
livestock_df = livestock_df[livestock_df['Item'].isin(lsu_data.keys())]
livestock_df['LSU_coeff'] = livestock_df.apply(get_lsu_value, axis=1)
livestock_df = livestock_df[livestock_df['Item'].isin(livestock_df[livestock_df['Element']=='Stocks'].Item.unique())]
livestock_df.loc[livestock_df['Unit'] == '1000 An', 'Value'] *= 1000
livestock_df.loc[livestock_df['Unit'] == '1000 An', 'Unit'] = 'An'
livestock_df['LSU'] = livestock_df['LSU_coeff'] *  livestock_df['Value']


### Transform fbs dataframe so each food item and flow type pair have a unique column for easy dasymetric distribution

In [8]:
# These are columns will be for distributing production and consumption onto their surrogates
fbs['ConsMCal'] = fbs['FoodSupplyMCal']
fbs['ProdMCal'] = fbs['CropProdMCal'] + fbs['AnimProdMCal']

# Sum all the foods over each country and food item
fbscatdf = fbs[['Area', 'Item'] + [col for col in fbs.columns if col[-4:] == 'MCal']
               ].groupby(['Area', 'Item']).sum().reset_index()
fbscatdf = fbscatdf.pivot(index='Area', columns=['Item'], values=list(fbscatdf.columns)[2:])
fbscatdf.columns = ['{} {}'.format(col[0], col[1]) for col in fbscatdf.columns]
fbscatdf = fbscatdf.reset_index()
fbscatdf.rename(columns={'Area':'ISO3'}, inplace=True)
fbscatdf.replace({'ISO3': Names}, inplace=True)

remaining_food_items = fbs['Item'].unique()
removed_foods = [f for f in original_food_items if f not in remaining_food_items]
print(removed_foods, 'were removed')

# Make Latex Table
total_prod = fbs['FoodSupplyMCal'].sum()
table = (fbs.groupby('Item').sum()[['FoodSupplyMCal']]).reset_index()
table['% of Total Food Supply'] = table['FoodSupplyMCal'] / total_prod * 100
table['Total Food Supply (Calories)'] = table['FoodSupplyMCal'] * 10**6
crop_table = table.reset_index().sort_values(by='% of Total Food Supply',ascending=False)[['Item','Total Food Supply (Calories)','% of Total Food Supply']]
crop_table.head(10)


['Population', 'Grand Total', 'Vegetal Products', 'Animal Products', 'Cereals - Excluding Beer', 'Starchy Roots', 'Sugar Crops', 'Sugar & Sweeteners', 'Honey', 'Pulses', 'Treenuts', 'Oilcrops', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine', 'Stimulants', 'Spices', 'Alcoholic Beverages', 'Alcohol, Non-Food', 'Meat', 'Offals', 'Animal fats', 'Fish, Seafood', 'Miscellaneous', 'Infant food'] were removed


,Item,Total Food Supply (Calories),% of Total Food Supply
92,Wheat and products,1.423272e+15,18.295741
71,Rice and products,1.390955e+15,17.880307
81,Sugar (Raw Equivalent),5.604411e+14,7.204303
39,Maize and products,4.063323e+14,5.223280
43,Milk - Excluding Butter,3.903971e+14,5.018438
62,Pigmeat,2.387685e+14,3.069297
91,"Vegetables, other",2.150499e+14,2.764402
78,Soyabean Oil,1.962909e+14,2.523261
12,Cassava and products,1.626902e+14,2.091334
66,Potatoes and products,1.591138e+14,2.045360


### Data Validation

In [9]:
# # energy_cols = ['Item', 'CropProdMCal', 'AnimProdMCal', 'ProcProdMCal', 'FeedMCal', 'FoodMCal',
# #        'ProcMCal', 'LossMCal', 'ResiMCal', 'SeedMCal', 'StocMCal', 'TourMCal',
# #        'ImpoMCal', 'ExpoMCal', 'FoodSupplyMCal', 'OtherMCal', 'pop2015',
# #        'pop2021', 'ConsMCal', 'ProdMCal']

# # food_flows = fbs[energy_cols].groupby(['Item']).sum()
# # food_flows['diff'] = food_flows['ProdMCal'] - food_flows['FoodSupplyMCal']
# # food_flows.sort_values(by='diff').head(10)

# for food in foods:
#     df = fbs[fbs['Item'] == food].sum()
#     #sua_balance[food] = df['ProdMCal'] + df['ImpoMCal'] - df['ExpoMCal'] - df['StocMCal'] - (df['FoodMCal'] + df['FeedMCal'] + df['ProcMCal'] + df['OtherMCal'])
#     sua_balance[food] = df['ImpoMCal'] - df['ExpoMCal']
#     sua_balance[food] = sua_balance[food].item()
#     try:
#         assert(sua_balance[food] > 0)
#     except:
#         print(food, sua_balance[food])

for item in fbs.Item.unique():
    df = fbs[fbs['Item'] == item]
    total_prod = sum([df[col].sum() for col in ['CropProdMCal', 'ProcProdMCal', 'AnimProdMCal']])
    total_cons = sum([df[col].sum() for col in ['FoodSupplyMCal', 'FeedMCal', 'ProcMCal', 'OtherMCal']])
    epsilon = 1
    if item in ['Sugar (Raw Equivalent)', 'Sugar beet','Sugar cane', 'Sugar non-centrifugal', 'Sweeteners, Other']:
        print(item, 'prod: ', total_prod, 'Cons:', total_cons)
    try:
        assert(abs(total_prod - total_cons) < epsilon)
    except:
        print('Unbalanced:',item, round(total_prod), round(total_cons))

Unbalanced: Aquatic Animals, Others 638154 638176
Unbalanced: Beverages, Alcoholic 74037510 74050426
Unbalanced: Beverages, Fermented 10145379 10148581
Unbalanced: Cephalopods 2625191 2625209
Unbalanced: Crustaceans 7387426 7387430
Unbalanced: Demersal Fish 9350838 9350840
Unbalanced: Fish, Liver Oil 32903 33156
Unbalanced: Freshwater Fish 42534512 42534519
Unbalanced: Marine Fish, Other 6153213 6153214
Unbalanced: Molluscs, Other 4273941 4273943
Sugar (Raw Equivalent) prod:  637934440.0000001 Cons: 637934440.0
Sugar beet prod:  168544600.0 Cons: 168544600.0
Sugar cane prod:  562347000.0 Cons: 562347000.0
Sugar non-centrifugal prod:  34440120.0 Cons: 34440120.0
Sweeteners, Other prod:  347820000.0 Cons: 347820000.0


In [10]:
fbs.Item.unique()

array(['Apples and products', 'Aquatic Animals, Others', 'Aquatic Plants',
       'Aquatic Products, Other', 'Bananas', 'Barley and products',
       'Beans', 'Beer', 'Beverages, Alcoholic', 'Beverages, Fermented',
       'Bovine Meat', 'Butter, Ghee', 'Cassava and products',
       'Cephalopods', 'Cereals, Other', 'Citrus, Other', 'Cloves',
       'Cocoa Beans and products', 'Coconut Oil', 'Coconuts - Incl Copra',
       'Coffee and products', 'Cottonseed', 'Cottonseed Oil', 'Cream',
       'Crustaceans', 'Dates', 'Demersal Fish', 'Eggs',
       'Fats, Animals, Raw', 'Fish, Body Oil', 'Fish, Liver Oil',
       'Freshwater Fish', 'Fruits, other', 'Grapefruit and products',
       'Grapes and products (excl wine)', 'Groundnut Oil', 'Groundnuts',
       'Lemons, Limes and products', 'Maize Germ Oil',
       'Maize and products', 'Marine Fish, Other',
       'Meat, Aquatic Mammals', 'Meat, Other', 'Milk - Excluding Butter',
       'Millet and products', 'Molluscs, Other', 'Mutton & Goat M

### Make Tables

In [11]:
total_prod = fbs['CropProdMCal'].sum()
total_cons = fbs['FoodSupplyMCal'].sum()
crop_fbs = fbs[fbs['Class'] == 'Crop']
table = (crop_fbs.groupby('Item').sum()[['FoodSupplyMCal','CropProdMCal']]).reset_index()
table['% of Total Food Supply'] = table['FoodSupplyMCal'] / total_cons * 100
table['% of Total Production'] = table['CropProdMCal'] / total_prod * 100
table['Total Food Supply (Calories)'] = table['FoodSupplyMCal'] * 10**6
table['Total Production (Calories)'] = table['CropProdMCal'] * 10**6
full_table = table.reset_index().sort_values(by='% of Total Production',ascending=False)[['Item','Total Food Supply (Calories)','% of Total Food Supply','Total Production (Calories)','% of Total Production']]
crop_table = full_table[['Item','% of Total Food Supply', '% of Total Production']]
crop_table.head(10)

,Item,% of Total Food Supply,% of Total Production
25,Maize and products,5.223280,23.159742
47,Rice and products,17.880307,15.933176
68,Wheat and products,18.295741,15.850908
36,Palm kernels,0.000385,10.997368
55,Soyabeans,0.443832,6.594090
59,Sugar cane,0.144420,3.425117
2,Barley and products,0.244330,2.954918
46,Rape and Mustardseed,0.015604,2.162989
7,Cassava and products,2.091334,1.994471
43,Potatoes and products,2.045360,1.507894


In [12]:
total_proc_cons = fbs['FoodSupplyMCal'].sum()
proc_fbs = fbs[fbs['Processed'] == 'Yes']
table = (proc_fbs.groupby('Item').sum()[['FoodSupplyMCal']]).reset_index()
table['% of Total Food Supply'] = table['FoodSupplyMCal'] / total_proc_cons * 100
table['Total Food Supply (Calories)'] = table['FoodSupplyMCal'] * 10**6
full_table = table.reset_index().sort_values(by='% of Total Food Supply',ascending=False)[['Item','Total Food Supply (Calories)','% of Total Food Supply']]
proc_table = full_table[['Item','% of Total Food Supply']]
proc_table

,Item,% of Total Food Supply
21,Sugar (Raw Equivalent),7.204303
20,Soyabean Oil,2.523261
15,Palm Oil,1.843824
13,"Oilcrops Oil, Other",1.401920
23,Sunflowerseed Oil,1.148321
3,"Butter, Ghee",1.081413
17,Rape and Mustard Oil,1.075105
0,Beer,1.049269
7,"Fats, Animals, Raw",1.038907
24,"Sweeteners, Other",0.902713


## 2. Grid Data Processing

### The grid will include crop production from 2015, livestock counts from 2006, and fish catch data from 2015

In [13]:
import xarray as xr
import glob

###########################################################################
#                                STEP 1                                   #
#                                                                         #
#                   Import Each Grid Variable Netcdf                      #
#                                                                         #
###########################################################################
## Accidentally messed up the paths with the config files
# POPULATION AND GRID AREA
pop_nc_path = config['paths']['population.nc']
ds = xr.load_dataset(pop_nc_path)

# PRODUCTION DATA
netcdf_paths = os.path.join(config['paths']['crop_prod_dir'], '*.nc')
path_list = sorted(glob.glob(netcdf_paths))

gaez_crop_prod_T = np.zeros(ds.grid_area.to_numpy().shape)
for prod_path in path_list:
    crop_name = prod_path.split('/')[-1].split('.')[0]
    #print(crop_name)
    crop_ds = xr.open_dataset(prod_path)
    crop_ds = crop_ds.reindex(y=list(reversed(crop_ds.y)))
    crop_da = crop_ds.band_data.sel(band=1).drop_vars('band').rename({'y':'lat','x':'lon'})
    ds = ds.assign({crop_name: (crop_da.dims, 1000 * crop_da.values)})
    gaez_crop_prod_T += ds[crop_name].to_numpy()
ds['total_crop_prod'] = sum([ds[crop_prod] for crop_prod in list(ds.data_vars)[4:]])
 
# GRIDDED LIVESTOCK DATA
netcdf_paths = os.path.join(config['paths']['livestock_dir'], '*.nc')
path_list = sorted(glob.glob(netcdf_paths))

for livestock_path in path_list:
    livestock_name = livestock_path.split('/')[-1].split('.')[0]
    livestock_ds = xr.load_dataset(livestock_path)
    livestock_da = livestock_ds[livestock_name]
    livestock_da = livestock_da.where(livestock_da > 1000)
    ds = ds.assign(**{livestock_name: livestock_da})

# Make surrogates for combinations of livestock
ds['DairyProducers'] = sum([ds[An].fillna(0) for An in ['Bf','Ct','Sh','Gt']])
ds['PoultryProducers'] = sum([ds[An].fillna(0) for An in ['Dk','Ch']])
ds['total_livestock'] = ds['Bf'] + ds['Ct'] + ds['Sh'] + ds['Gt'] + ds['Dk'] + ds['Ch']+ ds['Pg']
 
# GRIDDED FISH CATCH DATA
fish_nc_path = config['paths']['fish_catch.nc']
fish_ds = xr.load_dataset(fish_nc_path)
ds['catch_pel'] = fish_ds['catch_pel']
ds['catch_dem'] = fish_ds['catch_dem']
ds['total_catch'] = fish_ds['catch_dem'] + fish_ds['catch_pel']

# Rescale catch data to be between 0 and 1 for easy global distribution
ds['total_catch_frac'] = ds['total_catch'] / ds['total_catch'].sum()
ds['catch_dem_frac'] = ds['catch_dem'] / ds['catch_dem'].sum()
ds['catch_pel_frac'] = ds['catch_pel'] / ds['catch_pel'].sum()

### Now dasymetrically map the country data into the grid cells

#### First we dasymmetrically donwscale animal counts, and animal masses from FAO 2022 Livestock data

In [14]:
sys.path.append(config['sesame_path'])# Once sesametoolbox is installable through pypl this line will be uncessary
import sesametoolbox as st

abbrev_to_livestock = {'Bf': 'Buffalo', 'Ch': 'Chickens', 'Ct': 'Cattle', 'Gt': 'Goats', 'Pg': 'Swine / pigs', 'Sh': 'Sheep'}

new_lsu_df = pd.DataFrame(list(fao_country_to_region.items()), columns=['ISO3', 'Region']).merge(lsu_df, on='Region')  
for animal_key, animal_name in abbrev_to_livestock.items():
    ds[animal_key + 'LSU'] = st.table_2_grid(animal_key, 'LSU', input_ds=ds, input_df=livestock_df[livestock_df['Item'] == animal_name])['LSU']

ds['total_livestockLSU'] = sum([ds[AnLSU].fillna(0) for AnLSU in ['BfLSU','ChLSU','CtLSU','GtLSU','PgLSU','ShLSU']]) #'DkLSU',
ds['DairyProducersLSU'] = sum([ds[AnLSU].fillna(0) for AnLSU in ['BfLSU','CtLSU','GtLSU','ShLSU']])
ds['NonSpatial'] = xr.full_like(ds['total_livestock'], np.nan)

Distributing LSU onto Bf.
Distributing LSU onto Ch.
Distributing LSU onto Ct.
Distributing LSU onto Gt.
Distributing LSU onto Pg.
Distributing LSU onto Sh.


### Next Dasymmetrically Map National Basal Metabolic Rate onto Population

In [15]:
bmi_bmr_path = config['paths']['bmi_bmr.csv']
bmr_df = pd.read_csv(bmi_bmr_path)
bmr_df['total_bmr'] = bmr_df['approx_bmr'] * bmr_df['pop2015'] * 1000 #pop2015 is given in units of 1000 ppl
bmr_df['ISO3'] = bmr_df['ISO']

ds['pop2015'] = ds['population_count'].sel(time="2015-01-01T00:00:00.000000000")
ds['bmr'] = xr.zeros_like(ds['grid_area'])
ds['bmr'] = st.table_2_grid('pop2015', 'total_bmr', input_ds=ds.sel(time='2015'), input_df=bmr_df, verbose='True')['total_bmr']
ds['tmr'] = 5/3*ds['bmr'] / ds['grid_area']
ds['tmr_Cal_p_grid_cell'] = 5/3*ds['bmr']
ds['tmr'].attrs = {'units':'Calories per day per sqm',
                   'long_name':'Total Basal Metabolic Rate'}

Distributing total_bmr onto pop2015.
Global sum of jurisdictional dataset : 10609415147186.9
Global sum of gridded dataset : 10609415147186.902



### Next we distribute production, consumption, and feed from the 2015 FBS onto the gridded surrogate data

In [16]:
with open(config['paths']['food_to_surrogate.json']), 'r') as json_file:
    food_to_surrogate = json.load(json_file)

item_to_surrogate = {item: 'NonSpatial' for item in fbs['Item'].unique()}
item_to_surrogate.update(food_to_surrogate)
print('\nUncategorized items in fbs', set(food_to_surrogate.keys()) ^ set(fbs['Item'].unique()))

for food in list(fbs.Item.unique()):
    if item_to_surrogate[food] != 'NonSpatial':
        ds['fbs_prod_' + food] = st.table_2_grid(item_to_surrogate[food], 'ProdMCal ' + food, input_ds=ds, input_df=fbscatdf)['ProdMCal ' + food]
    ds['fbs_cons_' + food] = st.table_2_grid('pop2015', 'ConsMCal ' + food, input_ds=ds, input_df=fbscatdf)['ConsMCal ' + food]
    ds['fbs_feed_' + food] = st.table_2_grid('total_livestockLSU', 'FeedMCal ' + food, input_ds=ds, input_df=fbscatdf)['FeedMCal ' + food]
ds

SyntaxError: unmatched ')' (2621657482.py, line 1)

In [ ]:
ds.to_netcdf(os.path.join(data_dir, '9-30ds.nc'))
#ds = xr.load_dataset('9-30ds.nc')

### Marine and Fish Production

In [ ]:
marine_surrogates = {"Aquatic Animals, Others": "total_catch_frac",
                  "Aquatic Plants": "catch_dem_frac",
                  "Cephalopods": "total_catch_frac",
                  "Crustaceans": "catch_dem_frac",
                  "Demersal Fish": "catch_dem_frac",
                  "Marine Fish, Other": "total_catch_frac",
                  "Molluscs, Other": "catch_dem_frac",
                  "Pelagic Fish": "catch_pel_frac"}

for food in marine_surrogates.keys():
    ds['fbs_prod_' + food] = fbs[fbs['Item'] == food]['ProdMCal'].sum() * ds[marine_surrogates[food]]

pd.DataFrame(marine_surrogates, columns=['FBS Food', 'BoatsV2 Surrogate']).to_latex()

In [ ]:
excluded = ['Fish, Liver Oil', 'Pepper', 'Beverages, Fermented', 'Aquatic Products, Other', 'Sweeteners, Other', 'Fish, Body Oil', 'Beverages, Alcoholic', 'Meat, Aquatic Mammals']
total_prod = fbs['ProdMCal'].sum().item()
excluded_prod = fbs[fbs['Item'].isin(excluded)]['ProdMCal'].sum().item()
with_fresh = excluded_prod / total_prod * 100

(without_fresh - with_fresh) / without_fresh
sorted(excluded)

### Data Validation

In [ ]:
foods = [dv[9:] for dv in ds.data_vars if dv[:8] == 'fbs_prod']

for food in foods:
    grid_total = ds['fbs_prod_' + food].sum().item()
    fbs_total = fbs[fbs['Item'] == food]['ProdMCal'].sum().item()
    try:
        assert(round(grid_total - fbs_total) == 0)
    except:
        print(food, round(grid_total - fbs_total))

In [ ]:
fbs[['Item', 'ProdMCal']].groupby('Item').sum().reset_index()['ProdMCal'].sum().item() / 8e9 / 365

In [ ]:
for food in foods:
    df = fbs[fbs['Item'] == food].sum()
    sua_balance[food] = df['Production'] + df['Import Quantity'] - df['Export Quantity'] - df['Stock Variation'] - (df['Food'] + df['Feed'] + df['Seed'] + df['Losses'] + df['Processing'] + df['Other uses (non-food)'] + df['Tourist consumption'] + df['Residuals'])
    sua_balance[food] = sua_balance[food].item()
    try:
        assert(sua_balance[food] < 10)
    except:
        print(food, sua_balance[food])


In [ ]:
for type in ['Animal Products']:#fbs.Category.unique():
    foods_of_this_type = [food for food in foods if food in fbs[fbs.Category == type].Item.unique()]
    for food in foods_of_this_type:
        print(food, round(ds['fbs_prod_' + food].sum().item()) - round(ds['fbs_cons_' + food].sum().item()))
        print(food, fbs[fbs['Item'] == food]['ProdMCal'].sum().item() + fbs[fbs['Item'] == food]['ProcProdMCal'].sum().item() - fbs[fbs['Item'] == food]['FoodSupplyMCal'].sum().item())



In [ ]:
print(fbs['ImpoMCal'].sum().item())
print(fbs['ExpoMCal'].sum().item())

## 3. Data Visualization

### Plotting Functions

In [ ]:
import xarray as xr
ds = xr.load_dataset('temp_ds.nc')

In [ ]:
import matplotlib.colors as colors
import numpy as np
import seaborn as sns
from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import matplotlib.pyplot as plt

# Change the font size globally
plt.rcParams.update({'font.size': 14})

image_dir = '/Users/maxwellkaye/Documents/PhD Research/drafts/Farm2Metabolism/Images/'

def hist_da(da,  title='', xlabel='', ylabel='', savefig_path='', xscale='log', color=None, color_spectrum=False):
    data = da.values.flatten()
    data = data[~np.isnan(data)]
    
    if xscale == 'log':
        bins = np.logspace(np.log10(data[data > 0].min()), np.log10(data.max()), num=40)
    elif xscale == 'symlog':
        data = data[abs(data) > 1]
        logbins = np.logspace(np.log10(data[data > 0].min()), np.log10(data.max()), num=20)
        bins = np.append(-logbins[::-1],logbins)

    if color == None:
        color = sns.color_palette("muted")[3]
    
    #sns.histplot(data, bins=bins, color=red, alpha=0.5, stat="count", multiple="stack")
    hist_data = sns.histplot(data, bins=bins, color=color, alpha=0.7, edgecolor='black', linewidth=0.5)

    if color_spectrum:
        # Get the patches (bars) from the plot
        patches = hist_data.patches

        cmap = LinearSegmentedColormap.from_list("custom_cmap", [color, "dimgray"])

        # Loop through each bar and set its color darker as we go left
        for i, patch in enumerate(patches):
            # Normalize color to go from light to dark (index from the right, so reverse the i)
            color_value = i / len(patches)
            patch.set_facecolor(cmap(1 - color_value))  # 1-color_value for dark to light

    plt.xscale(xscale)  # Set x-axis to logarithmic scale
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig(savefig_path)
    plt.tight_layout()
    plt.show()

def plot_da(da, cmap, norm, save_to_path=None):
    # Create the plot
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.Robinson())
    ax.set_global()
        
    ax.coastlines(linewidth=0.2)
    for feature in [cfeature.BORDERS]:#, cfeature.LAND, cfeature.OCEAN]:
        ax.add_feature(feature, linewidth=0.2)
        
    def custom_formatter(x, pos):
        """Custom formatter function to format ticks as 1e4, 5e5, etc."""
        if x == 0:
            return "0"
        exponent = int(np.floor(np.log10(abs(x))))
        coeff = x / 10**exponent
        return f"{coeff:.0f}e{exponent}"
        
    pl = da.plot(transform=ccrs.PlateCarree(),
                 cmap=cmap,
                 norm=norm, 
                 cbar_kwargs={'orientation': 'horizontal',
                             #'ticks':[0, 1e11, 5e11, 1e12, 1e13, 5e13, 1e14],
                             #'format': custom_formatter,
                             'pad': 0.05,
                             'aspect': 50,
                             'shrink': 0.75,
                             'extend': 'min'})
    
    if save_to_path:
        plt.savefig(save_to_path, bbox_inches='tight', pad_inches=0)

    return ax

my_cmap = sns.color_palette("flare", as_cmap=True)
my_norm = colors.LogNorm(vmin=1e2, vmax=1e8)

### Figure 1

In [ ]:
import seaborn as sns
from matplotlib.colors import to_hex

fbsv = fbs.rename(columns={ 'Production': 'ProdkT',
                            'Processing': 'ProckT',
                            'Feed': 'FeedkT',
                            'Food': 'ConskT'})

fbsv['h2'] = fbsv['Type'].replace({'Fruits':'Fruits, Vegetables, Nuts', 'Vegetables':'Fruits, Vegetables, Nuts', 'Nuts':'Fruits, Vegetables, Nuts',
                                   'AlcoholicBeverages':'Spices, Sweeteners, and Beverages', 'StimulantsAndSpices':'Spices, Sweeteners, and Beverages',
                                   'SugarCrop': 'Sugar', 'Pulses':'Pulses, Roots, Tubers', 'Roots':'Pulses, Roots, Tubers',
                                   'Meat': 'Animal Products', 'NonMeatAnimalProduct': 'Animal Products', 'Seafood': 'Animal Products',
                                   'Oil': 'Oilcrops'})

#columns = ['CropProdMCal', 'AnimProdMCal', 'ProcProdMCal', 'FeedMCal', 'ProcMCal', 'FoodMCal', 'OtherMCal']
columns = [col for col in fbsv.columns if col[-4:] == 'MCal']

fbsv = fbsv[['Item', 'h2'] + columns]

fbsv = fbsv.groupby(['Item', 'h2']).sum().reset_index()
fbsv.rename(columns={'Item': 'h3'}, inplace=True)

# Add Metabolism Column
total_cons_supply_grid = sum([ds[dv].fillna(0) for dv in ds.data_vars if dv[:8] == 'fbs_cons']) #* 10**12 / ds['grid_area'] / 365  #6 for m to km, 6 for MC to C
metabolism = {}
for food in foods:
    metabolism[food] = (ds['fbs_cons_' + food].fillna(0) / total_cons_supply_grid.fillna(0) * ds['tmr_Cal_p_grid_cell'].fillna(0)).sum().item()

def map_metabololsim(food):
    if food in metabolism.keys():
        return metabolism[food]
    else:
        return 0

fbsv['Metabolism'] = fbsv['h3'].map(map_metabololsim)

# Add Lancet Diet column
lancet_diet = [
    ['Whole Grains', 'Cereals', 811],
    ['Starchy Tubers', 'Pulses, Roots, Tubers', 39],
    ['Vegetables', 'Fruits, Vegetables, Nuts', 78],
    ['Fruits','Fruits, Vegetables, Nuts', 126],
    ['Dairy', 'Animal Products', 153],
    ['Red Meat', 'Animal Products', 30],
    ['Poultry', 'Animal Products', 62],
    ['Eggs', 'Animal Products', 19],
    ['Fish', 'Animal Products', 40],
    ['Legumes', 'Pulses, Roots, Tubers', 284],
    ['Nuts', 'Fruits, Vegetables, Nuts', 291],
    ['Oils', 'Oilcrops', 450],
    ['Sugars', 'Sugar', 120]
]

fbsv['Lancet'] = 0
row = {col: 0 for col in fbsv.columns}
for food, category, calories in lancet_diet:
    row.update({'Lancet': calories, 'h3': food, 'h2': category})
    fbsv.loc[len(fbsv)] = row
    
# Adding colors based on category and food category
turquoise, orange, blue, pink, green, yellow, brown = sns.color_palette("Set2", len(fbsv['h2'].unique()))
cat_color_dict = {'Fruits, Vegetables, Nuts': green,
                'Animal Products': pink,
                'Cereals': yellow,
                'Pulses, Roots, Tubers': brown,
                'Spices, Sweeteners, and Beverages': blue,
                'Oilcrops': orange,
                'Sugar': turquoise}
# category_colors = sns.color_palette("Set2", len(fbsv['h2'].unique()))
#category_color_dict = dict(zip(fbsv['h2'].unique(), category_colors))
fbsv['color'] = fbsv['h2'].map(cat_color_dict)

# Convert RGB colors to hexadecimal
fbsv['color'] = fbsv['color'].apply(lambda x: to_hex(x, keep_alpha=False))
fbsv = pd.melt(fbsv, id_vars=['h2', 'h3', 'color'], var_name='h1', value_name='weight')
fbsv = fbsv[['h1', 'h2', 'h3', 'color', 'weight']]

# Vary colors
x = 0.2  # 0 means no change; 0.5 means large change

np.random.seed(42)
def adjust_color(color, x):
    import colorsys
    hslcol = np.array(colorsys.rgb_to_hls(*tuple(int(color[i:i+2], 16)/255 for i in (1, 3, 5))))
    levs = np.random.uniform(1-x, 1+x) * hslcol[2]
    hslcol[2] = levs
    rgbcol = tuple(min(max(round(i * 255), 0), 255) for i in colorsys.hls_to_rgb(*hslcol))
    return f'#{rgbcol[0]:02X}{rgbcol[1]:02X}{rgbcol[2]:02X}'

fbsv['color'] = fbsv.groupby('h3')['color'].transform(lambda color: adjust_color(color.iloc[0], x))

# Normalize weights to be percentages
voronoi_weights = {}
for h1val in columns:
    selected_rows = fbsv[fbsv['h1'] == h1val]
    total_weight = selected_rows['weight'].sum()
    if h1val[-4:] == 'MCal':
        print(h1val[:-4] + ':')
        C = total_weight * 10**6 / (8*10**9) / 365
        # pi * r**2 = A
        # A / pi = r**2
        # sqrt(A / pi) = r
        area_scale_factor = 139.69
        D = 2 * np.sqrt(C * area_scale_factor / np.pi)
        print('\t\t', round(C), 'Cal/person/day')
        print('\t\t', round(D), 'Voronoi Diameter')
        voronoi_weights[h1val[:-4]] = C
    fbsv.loc[selected_rows.index, 'weight'] = selected_rows['weight'] / total_weight * 100
    #fbsv.loc[selected_rows.index, 'weight'] = selected_rows['weight'] * 10**6 / (8*10**9) / 365

prod = voronoi_weights['CropProd'] + voronoi_weights['AnimProd'] + voronoi_weights['ProcProd']
cons = voronoi_weights['FoodSupply'] + voronoi_weights['Feed'] + voronoi_weights['Proc'] + voronoi_weights['Other']
assert(round(prod) == round(cons))

fbsv['weight'] = fbsv['weight'].round(5) 
fbsv.to_csv(os.path.join(data_dir,'food_data','fbsv.csv'))
fbsv

In [ ]:
#fbsv[fbsv['h1']=='Metabolism']['weight'].sum()

ds['tmr_Cal_p_grid_cell'].sum().item() / 8e9

In [ ]:
sum(fbs['FoodSupplyMCal']) / 365 / 8e9 * 10**6

In [ ]:
import seaborn as sns
from matplotlib.colors import to_hex

fbsv = fbs.rename(columns={ 'Production': 'ProdkT',
                            'Processing': 'ProckT',
                            'Feed': 'FeedkT',
                            'Food': 'ConskT'})

fbsv['h2'] = fbsv['Type'].replace({'Fruits':'Fruits, Vegetables, Nuts', 'Vegetables':'Fruits, Vegetables, Nuts', 'Nuts':'Fruits, Vegetables, Nuts',
                                   'AlcoholicBeverages':'Spices, Sweeteners, and Beverages', 'StimulantsAndSpices':'Spices, Sweeteners, and Beverages',
                                   'SugarCrop': 'Sugar', 'Pulses':'Pulses, Roots, Tubers', 'Roots':'Pulses, Roots, Tubers',
                                   'Meat': 'Animal Products', 'NonMeatAnimalProduct': 'Animal Products', 'Seafood': 'Animal Products',
                                   'Oil': 'Oilcrops'})

#columns = ['CropProdMCal', 'AnimProdMCal', 'ProcProdMCal', 'FeedMCal', 'ProcMCal', 'FoodMCal', 'OtherMCal']
columns = [col for col in fbsv.columns if col[-4:] == 'MCal']

fbsv = fbsv[['Item', 'h2'] + columns]

fbsv = fbsv.groupby(['Item', 'h2']).sum().reset_index()
fbsv.rename(columns={'Item': 'h3'}, inplace=True)







### Figure 2

In [ ]:
thresh = 300
my_cmap = sns.color_palette("flare", as_cmap=True)
my_norm = colors.LogNorm(vmin=thresh, vmax=1e7)
my_cmap.set_bad(colo='white')

has_no_data_mask = xr.full_like(ds['grid_area'], fill_value=True, dtype=bool)
for da in [ds[dv] for dv in ds.data_vars if dv[:8] == 'fbs_prod']:
    has_no_data_mask &= np.isnan(da)
has_data_mask = ~has_no_data_mask

prod_mcal_p_year = sum([ds[dv].fillna(0) for dv in ds.data_vars if dv[:8] == 'fbs_prod'])
prod_p_sqm_p_day_da = prod_mcal_p_year * 10**12 / ds['grid_area'] / 365  #6 for m to km, 6 for MC to C
min_thresh_mask = prod_p_sqm_p_day_da.to_numpy() > thresh

mask = has_data_mask & min_thresh_mask
prodda = prod_p_sqm_p_day_da.where(prod_p_sqm_p_day_da.to_numpy() > 1e-10) # save pre-masked data for histogram
prod_p_sqm_p_day_da = prod_p_sqm_p_day_da.where(mask, other=np.nan)

prod_p_sqm_p_day_da.attrs = {'long_name': 'Total Production', 'units': 'Calories / sqkm / day'}
plot_da(prod_p_sqm_p_day_da, cmap=my_cmap, norm=my_norm, save_to_path=image_dir+'total_prod_map.png')

In [ ]:
hist_da(prodda, title='Distribution of Caloric Production', xlabel='Calories Production per sqkm per day', ylabel='Number of Grid Cells', savefig_path=image_dir+'prod_dist_hist.png')

### Figure 3

In [ ]:
def create_diverging_cmap(start_color, end_color, white_percentile=0.5):
    start_color = colors.to_rgb(start_color)
    end_color = colors.to_rgb(end_color)
    # Create a dictionary to define the colormap
    cdict = {
        'red':   [[0.0, start_color[0], start_color[0]],
                  [white_percentile, 1.0, 1.0],
                  [1.0, end_color[0], end_color[0]]],

        'green': [[0.0, start_color[1], start_color[1]],
                  [white_percentile, 1.0, 1.0],
                  [1.0, end_color[1], end_color[1]]],

        'blue':  [[0.0, start_color[2], start_color[2]],
                  [white_percentile, 1.0, 1.0],
                  [1.0, end_color[2], end_color[2]]]
    }
    custom_cmap = colors.LinearSegmentedColormap('CustomMap', segmentdata=cdict, N=256)
    return custom_cmap

ds['maxf'] = xr.zeros_like(ds['grid_area'])
ds['netf'] = xr.zeros_like(ds['grid_area'])

foods = [dv[9:] for dv in ds.data_vars if dv[:8] == 'fbs_prod']

fbs['Category'] = fbs['Type'].replace({'Fruits':'Fruits, Vegetables, Nuts', 'Vegetables':'Fruits, Vegetables, Nuts', 'Nuts':'Fruits, Vegetables, Nuts',
                                   'AlcoholicBeverages':'Spices, Sweeteners, and Beverages', 'StimulantsAndSpices':'Spices, Sweeteners, and Beverages',
                                   'SugarCrop': 'Sugar', 'Pulses':'Pulses, Roots, Tubers', 'Roots':'Pulses, Roots, Tubers',
                                   'Meat': 'Animal Products', 'NonMeatAnimalProduct': 'Animal Products', 'Seafood': 'Animal Products',
                                   'Oil': 'Oilcrops'})
turquoise, orange, blue, pink, green, yellow, brown = sns.color_palette("Set2", len(fbs['Category'].unique()))

cat_color_dict = {'Fruits, Vegetables, Nuts': green,
                'Animal Products': pink,
                'Cereals': yellow,
                'Pulses, Roots, Tubers': brown,
                'Spices, Sweeteners, and Beverages': blue,
                'Oilcrops': orange,
                'Sugar': turquoise}

for type in fbs.Category.unique():
    ds['netf_'+type] = xr.zeros_like(ds['grid_area'])
    foods_of_this_type = [food for food in foods if food in fbs[fbs.Category == type].Item.unique()]
    for food in foods_of_this_type:
        ds['netf_'+type] += (ds['fbs_prod_' + food].fillna(0) - ds['fbs_cons_' + food].fillna(0)) / ds['grid_area'] / 365 * 10**12  #+ ds['fbs_feed_' + food].fillna(0))
        #ds['prod_'+type] += ds['fbs_prod_' + food].fillna(0)
    
    ds['netf_'+type].attrs = {
        'long_name': 'Net Caloric Surplus of ' + type,
        'units': 'Calories / sqkm / day',
        'description': 'This variable contains total caloric surplus/deficit for each grid cell'
    }        

    my_cmap = create_diverging_cmap('black', cat_color_dict[type])
    my_norm = colors.Normalize(vmin=-1e4, vmax=1e4)
    plot_da(ds['netf_'+type], cmap=my_cmap, norm=my_norm, save_to_path=image_dir+'nf_'+type+'_map.png')
    

In [ ]:
info = {}
for type in fbs.Category.unique():
    print(type, "{:.2e}".format(ds['netf_'+type].where(ds['netf_'+type].values>0).sum().item()), "{:.2e}".format(ds['netf_'+type].where(ds['netf_'+type].values<0).sum().item()))
    #print(type, "{:.2e}".format(ds['netf_'+type].sum().item()))

In [ ]:
2.05e9 / 8e9

### Figure 4

In [ ]:
for type in fbs.Category.unique():
    hist_da(ds['netf_'+type], color=cat_color_dict[type], title=type + ' Net Caloric Balance Distribution', xlabel = 'Energy Balance [Calories / sqkm / day]', ylabel = 'Number of Grid Cells', xscale='symlog', color_spectrum=True, savefig_path = image_dir + type + '_balance_dist.png')
            

In [ ]:
from scipy import stats

#fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8), gridspec_kw={'width_ratios': [1.5, 1]})


###########################################################################
#                                                                         #
#                         Read in Demographic Data                        #
#                                                                         #
###########################################################################

bmr = pd.read_csv('/Users/maxwellkaye/Documents/PhD Research/data/food_data/BMI_BMR.csv')
fbs['ISO'] = fbs['Area'].apply(lambda s: Names[s])
fbs[['ISO','FoodSupplyMCal']].groupby('ISO').sum()
df = fbs[['ISO','FoodSupplyMCal']].groupby('ISO').sum().reset_index()

df = df.merge(bmr[['ISO','approx_bmr']], on='ISO')
pop_df = fbs[['Area','pop2015']].groupby('Area').mean().reset_index()
pop_df['ISO'] = pop_df['Area'].apply(lambda s: Names[s])
df = df.merge(pop_df, on='ISO')

df['total_metabolic_rate'] = df['approx_bmr'] * 5 / 3
df['CalpPersonpDay'] = df['FoodSupplyMCal'] / df['pop2015'] * 10**6 / 365

# Drop Duplicates form Region Classification
df = df[~df['Area'].isin(['Micronesia (Federated States of)', 'China'])]

# Include GDP from World Bank 2015
gdp = pd.read_csv('/Users/maxwellkaye/Documents/PhD Research/data/world_bank_gdp.csv')
df = df.merge(gdp[['Country Code', '2015']], left_on='ISO', right_on='Country Code', how='left').drop(columns=['Country Code'])
df = df.rename(columns={'2015':'2015 GDP'})

# Make a column that is empty strings for small countires, but keeps ISO for others
def kill_small_ctrys(iso):
    if df[df['ISO'] == iso]['pop2015'].item() < 1e8:
        return ''
    return iso
df['ISO_Large_Pop_Only'] = df['ISO'].map(kill_small_ctrys)

###########################################################################
#                                                                         #
#                             Set up the Plot                             #
#                                                                         #
###########################################################################

# Parameters
header1 = 'total_metabolic_rate'
header2 = 'CalpPersonpDay'
xlabel = 'Average National Total Metabolic Rate cal/person/day'
ylabel = 'Average National Food Supply (cal/person/day)'
title = 'Food Supply Vs. BMI Derived Average Metabolic Rate'

x = df[header1]
y = df[header2]
mask = ~np.isnan(x) & ~np.isnan(y) & np.isfinite(x) & np.isfinite(y)
x = x[mask]
y = y[mask]
gdp = (df['2015 GDP'] / df['pop2015'])[mask]
pop = df['pop2015'][mask]

my_norm = colors.LogNorm()# colors.TwoSlopeNorm(vmin=gdp.min()*.01, vcenter=gdp.mean(), vmax=gdp.max())
my_cmap = sns.color_palette("viridis", as_cmap=True)
scatter = ax1.scatter(x, y, c=gdp, cmap=my_cmap, norm=my_norm, s=pop/1e5, alpha=.8)

###########################################################################
#                                                                         #
#               Plot the line of best fit and 1 to 1 line                 #
#                                                                         #
###########################################################################

slope, intercept, r_value, best_fit_p_value, std_err = stats.linregress(x, y)
best_line = slope * x + intercept
ax1.plot(x, best_line, color='black', linestyle='--', linewidth=.8,label=f'Unweighted Best Fit (R={r_value:.2f}, m={slope:.2f})')
ax1.plot(x, x, color='black', label=f'One to One Line')

###########################################################################
#                                                                         #
#               Plot the population weighted best fit line                #
#                                                                         #
###########################################################################
w = df['pop2015'][mask]
w_mean_x = np.average(x, weights=w)
w_mean_y = np.average(y, weights=w)

# Weighted slope and intercept
slope = np.sum(w * (x - w_mean_x) * (y - w_mean_y)) / np.sum(w * (x - w_mean_x)**2)
intercept = w_mean_y - slope * w_mean_x

# Calculate the regression line
line = slope * x + intercept

# Calculate the R-squared value for the weighted regression
y_pred = slope * x + intercept
ss_res = np.sum(w * (y - y_pred)**2)
ss_tot = np.sum(w * (y - w_mean_y)**2)
r_value = np.sqrt(1 - ss_res / ss_tot)

# Plot the weighted regression line
ax1.plot(x, line, color='black',  linestyle=':', label=f'Population Weighted Fit (R={r_value:.2f}, m={slope:.2f})')

###########################################################################
#                                                                         #
#               Annotate, Label axes, Title, Save, and Show               #
#                                                                         #
###########################################################################
for i, cntry_name in df['ISO_Large_Pop_Only'][mask].items():
    ax1.annotate(cntry_name, (x[i], y[i]))

ax1.set_xlabel(xlabel)
ax1.set_ylabel(ylabel)
ax1.set_title(title)
ax1.legend()

###########################################################################
#                                                                         #
#               Plot GDP vs Population Weighted Fit                       #
#                                                                         #
###########################################################################

loggdp = np.log10(gdp)

x = best_line
y = df['CalpPersonpDay']

maskp = ~np.isnan(y-x) & ~np.isnan(loggdp) & np.isfinite(y-x) & np.isfinite(loggdp)
xp = (loggdp)[maskp] # gdp
yp = (y-x)[maskp]    # residuals
pop = df['pop2015'][maskp]

for i, cntry_name in df['ISO_Large_Pop_Only'][maskp].items():
    ax2.annotate(cntry_name, (xp[i], yp[i]))

slope, intercept, r_value, residual_p_value, std_err = stats.linregress(xp, yp)
line = slope * xp + intercept
cbar = plt.colorbar(scatter, label='GDP per Capita')
scatter = ax2.scatter(xp,yp, s=pop/1e5, cmap=my_cmap, norm=my_norm, alpha=.8) #c=gdp[maskp]


#ax2.plot(xp, line, color='black', label=f'Best Fit (R={r_value:.2f}, m={slope:.2f})')
ax2.plot(xp, np.zeros(len(xp)), color='black', linestyle=':', label='Zero Line')
ax2.set_ylabel('Residual Calories to the Best Fit (kcal/person/day)')
ax2.set_xlabel('Log GDP Per Capita')
ax2.set_title(' GDP per Capita vs Best Fit Residual Food Supply Calories')

#plt.legend()
plt.legend(loc='lower left')#, bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.savefig(image_dir + 'metabolism_food_supply_with_residual_vs_gdp.png')
plt.show()
print('Best fit p value: ', best_fit_p_value)
print('Residual correlatipn r^2 vlaue: ', r_value**2)

### Figure 5

In [ ]:
thresh = 300
my_cmap = sns.color_palette("flare", as_cmap=True)
my_norm = colors.LogNorm(vmin=thresh, vmax=1e7)
my_cmap.set_bad(color='white')


tmrda = ds['tmr'] * 10**6  #10^6 for m to km
tmrda.attrs = {'long_name': 'Total Metabolic Rate', 'units': 'Calories / sqkm / day'}

plot_da(tmrda, cmap=my_cmap, norm=my_norm, save_to_path=image_dir + 'metabolism_map.png')

In [ ]:


hist_da(tmrda, title='Distribution of Human Metabolism', xlabel='Calories Metabolized per sqkm per day', ylabel='Number of Grid Cells', savefig_path=image_dir+'cons_dist_hist.png')


In [ ]:
# %reload_ext autoreload
# %autoreload 2
# sys.path.append('/Users/maxwellkaye/Documents/Phd Research/SESAME/src')

# import sesametoolbox as st

# food = 'Beans'
# fbscatdf['ProdMCal ' + food].sum()
# ds[item_to_surrogate[food]].sum()
# da = st.table_2_grid(item_to_surrogate[food], 'ProdMCal ' + food, input_ds=ds, input_df=fbscatdf)['ProdMCal ' + food]
# da
#ds.to_netcdf('temp_ds.nc')

In [ ]:
(ds['tmr'] * ds['grid_area']).sum()